## Preprocessing

In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop('IS_SUCCESSFUL', axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 352       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 403 (1.57 KB)
Trainable params: 403 (1.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
#Create a callback that saves the models weights every 5 epochs
checkpoint_path = "models/model_1/weights.hdf5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5)


In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
  1/804 [..............................] - ETA: 7s - loss: 0.6593 - accuracy: 0.5938
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
 10/804 [..............................] - ETA: 5s - loss: 0.5584 - accuracy: 0.7094
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
 20/804 [..............................] - ETA: 5s - loss: 0.5378 - accuracy: 0.7375
Epoch 1: saving model to models/model_1\weights.hdf5
 27/804 [>.............................] - ETA: 5s - loss: 0.5431 - accuracy: 0.7326
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
 35/804 [>.............................] - ETA: 5s - loss: 0.5479 - accuracy: 0.7295
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
 45/804 [>.............................] - ETA: 5s - loss: 0.5462 - accuracy: 0.72

368/804 [============>.................] - ETA: 3s - loss: 0.5434 - accuracy: 0.7362
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
375/804 [============>.................] - ETA: 3s - loss: 0.5431 - accuracy: 0.7362
Epoch 1: saving model to models/model_1\weights.hdf5
382/804 [=============>................] - ETA: 3s - loss: 0.5428 - accuracy: 0.7363
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
390/804 [=============>................] - ETA: 3s - loss: 0.5434 - accuracy: 0.7358
Epoch 1: saving model to models/model_1\weights.hdf5
396/804 [=============>................] - ETA: 3s - loss: 0.5435 - accuracy: 0.7358
Epoch 1: saving model to models/model_1\weights.hdf5
404/804 [==============>...............] - ETA: 3s - loss: 0.5421 - accuracy: 0.7368
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
410/804 [=

735/804 [==========================>...] - ETA: 0s - loss: 0.5420 - accuracy: 0.7354
Epoch 1: saving model to models/model_1\weights.hdf5
741/804 [==========================>...] - ETA: 0s - loss: 0.5422 - accuracy: 0.7352
Epoch 1: saving model to models/model_1\weights.hdf5
749/804 [==========================>...] - ETA: 0s - loss: 0.5425 - accuracy: 0.7348
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
755/804 [===========================>..] - ETA: 0s - loss: 0.5426 - accuracy: 0.7346
Epoch 1: saving model to models/model_1\weights.hdf5

Epoch 1: saving model to models/model_1\weights.hdf5
765/804 [===========================>..] - ETA: 0s - loss: 0.5420 - accuracy: 0.7351
Epoch 1: saving model to models/model_1\weights.hdf5
771/804 [===========================>..] - ETA: 0s - loss: 0.5419 - accuracy: 0.7349
Epoch 1: saving model to models/model_1\weights.hdf5
777/804 [===========================>..] - ETA: 0s - loss: 0.542

306/804 [==========>...................] - ETA: 3s - loss: 0.5396 - accuracy: 0.7363
Epoch 2: saving model to models/model_1\weights.hdf5
312/804 [==========>...................] - ETA: 3s - loss: 0.5394 - accuracy: 0.7366
Epoch 2: saving model to models/model_1\weights.hdf5
318/804 [==========>...................] - ETA: 3s - loss: 0.5391 - accuracy: 0.7365
Epoch 2: saving model to models/model_1\weights.hdf5

Epoch 2: saving model to models/model_1\weights.hdf5
326/804 [===========>..................] - ETA: 3s - loss: 0.5388 - accuracy: 0.7371
Epoch 2: saving model to models/model_1\weights.hdf5

Epoch 2: saving model to models/model_1\weights.hdf5
336/804 [===========>..................] - ETA: 3s - loss: 0.5387 - accuracy: 0.7370
Epoch 2: saving model to models/model_1\weights.hdf5
343/804 [===========>..................] - ETA: 3s - loss: 0.5397 - accuracy: 0.7360
Epoch 2: saving model to models/model_1\weights.hdf5

Epoch 2: saving model to models/model_1\weights.hdf5
351/804 [=

671/804 [========================>.....] - ETA: 1s - loss: 0.5415 - accuracy: 0.7345
Epoch 2: saving model to models/model_1\weights.hdf5

Epoch 2: saving model to models/model_1\weights.hdf5
681/804 [========================>.....] - ETA: 0s - loss: 0.5414 - accuracy: 0.7347
Epoch 2: saving model to models/model_1\weights.hdf5
687/804 [========================>.....] - ETA: 0s - loss: 0.5413 - accuracy: 0.7350
Epoch 2: saving model to models/model_1\weights.hdf5
695/804 [========================>.....] - ETA: 0s - loss: 0.5411 - accuracy: 0.7348
Epoch 2: saving model to models/model_1\weights.hdf5

Epoch 2: saving model to models/model_1\weights.hdf5
701/804 [=========================>....] - ETA: 0s - loss: 0.5414 - accuracy: 0.7348
Epoch 2: saving model to models/model_1\weights.hdf5
710/804 [=========================>....] - ETA: 0s - loss: 0.5415 - accuracy: 0.7347
Epoch 2: saving model to models/model_1\weights.hdf5

Epoch 2: saving model to models/model_1\weights.hdf5
716/804 [=

...

176/804 [=====>........................] - ETA: 4s - loss: 0.5382 - accuracy: 0.7402
Epoch 47: saving model to models/model_1\weights.hdf5

Epoch 47: saving model to models/model_1\weights.hdf5
186/804 [=====>........................] - ETA: 4s - loss: 0.5392 - accuracy: 0.7384
Epoch 47: saving model to models/model_1\weights.hdf5

Epoch 47: saving model to models/model_1\weights.hdf5
196/804 [======>.......................] - ETA: 4s - loss: 0.5405 - accuracy: 0.7371
Epoch 47: saving model to models/model_1\weights.hdf5

Epoch 47: saving model to models/model_1\weights.hdf5
206/804 [======>.......................] - ETA: 4s - loss: 0.5421 - accuracy: 0.7359
Epoch 47: saving model to models/model_1\weights.hdf5
215/804 [=======>......................] - ETA: 4s - loss: 0.5426 - accuracy: 0.7355
Epoch 47: saving model to models/model_1\weights.hdf5

Epoch 47: saving model to models/model_1\weights.hdf5
221/804 [=======>......................] - ETA: 4s - loss: 0.5433 - accuracy: 0.7346


556/804 [===================>..........] - ETA: 1s - loss: 0.5433 - accuracy: 0.7336
Epoch 47: saving model to models/model_1\weights.hdf5
565/804 [====================>.........] - ETA: 1s - loss: 0.5438 - accuracy: 0.7332
Epoch 47: saving model to models/model_1\weights.hdf5

Epoch 47: saving model to models/model_1\weights.hdf5
571/804 [====================>.........] - ETA: 1s - loss: 0.5439 - accuracy: 0.7332
Epoch 47: saving model to models/model_1\weights.hdf5

Epoch 47: saving model to models/model_1\weights.hdf5
581/804 [====================>.........] - ETA: 1s - loss: 0.5438 - accuracy: 0.7333
Epoch 47: saving model to models/model_1\weights.hdf5
588/804 [====================>.........] - ETA: 1s - loss: 0.5433 - accuracy: 0.7335
Epoch 47: saving model to models/model_1\weights.hdf5

Epoch 47: saving model to models/model_1\weights.hdf5
596/804 [=====================>........] - ETA: 1s - loss: 0.5424 - accuracy: 0.7342
Epoch 47: saving model to models/model_1\weights.hdf5



137/804 [====>.........................] - ETA: 5s - loss: 0.5362 - accuracy: 0.7409
Epoch 48: saving model to models/model_1\weights.hdf5
146/804 [====>.........................] - ETA: 4s - loss: 0.5365 - accuracy: 0.7399
Epoch 48: saving model to models/model_1\weights.hdf5

Epoch 48: saving model to models/model_1\weights.hdf5
152/804 [====>.........................] - ETA: 4s - loss: 0.5358 - accuracy: 0.7399
Epoch 48: saving model to models/model_1\weights.hdf5

Epoch 48: saving model to models/model_1\weights.hdf5
162/804 [=====>........................] - ETA: 4s - loss: 0.5335 - accuracy: 0.7407
Epoch 48: saving model to models/model_1\weights.hdf5
171/804 [=====>........................] - ETA: 4s - loss: 0.5331 - accuracy: 0.7418
Epoch 48: saving model to models/model_1\weights.hdf5

Epoch 48: saving model to models/model_1\weights.hdf5
177/804 [=====>........................] - ETA: 4s - loss: 0.5341 - accuracy: 0.7406
Epoch 48: saving model to models/model_1\weights.hdf5




Epoch 48: saving model to models/model_1\weights.hdf5
537/804 [===================>..........] - ETA: 1s - loss: 0.5442 - accuracy: 0.7302
Epoch 48: saving model to models/model_1\weights.hdf5


KeyboardInterrupt: 

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5548 - accuracy: 0.7269 - 650ms/epoch - 2ms/step
Loss: 0.554763913154602, Accuracy: 0.7268804907798767


In [19]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity.h5')